## Python script to generate CRC for dsPIC33E flash

### Summary
The dsPIC uses 24bit (3bytes) wide instructions. The XC16 compiler generated HEX files are following INTEL byte standard and puts "phantom" 00 bytes to the 4th place. However the Microchip classB library CRC function *CLASSB_CRCFlashTest* does not calulate the phantom byte. (To save some calculation time.) Therefore most of the CRC calculators on the PC side just does not have proper configurability to give the same result as the ClassB library.

The below script calculates the CRC according the Microchip ClassB library implementation.

#### Readings:
* Microchip ClassB: https://www.microchip.com/design-centers/home-appliance/class-b-safety-software
* dsPIC33 architecture: https://microchipdeveloper.com/dsp0201:dspic-dsc-architecture-review
* INTEL HEX format: https://en.wikipedia.org/wiki/Intel_HEX

### Install dependencies
First install crcmod if needed:
* *pip install crcmod*
* *pip install intelhex*
### Contact
mark.wendler@microchip.com

Create CRC calculator function according to the default ClassB CRC calculation function. (Polynomal is 0x8005 by default in Microchip implementation)

In [2]:
import crcmod
crc16_func = crcmod.mkCrcFun(0x18005, rev=False, initCrc=0xFFFF, xorOut=0x0000)

Just a short test to verify crccalc functionality

In [3]:
word24bitIn = bytearray((0xe0, 0x00 , 0x00, 0x32, 0x00, 0x02));

result = crc16_func(word24bitIn);

hex(result)

'0x83d1'

Install intelHex if needed: pip install intelhex

In [4]:
from intelhex import IntelHex16bit #Load intelHex
ih = IntelHex16bit("input.hex") #Open file

Create a function to extract hex file and format the data as crc calc requires.

In [5]:
def ExtractHexData(baseAddr, length, intelHex):
    """Rearrange memory map from intelHex to byte array: [Upper,Higher,Lower,Upper,Higher,Lower] This format cen be used then for CRC calculation

    Args:
        baseAddr (int): Starting address to extract from hex file (dsPIC program counter address)
        length   (int): The length address of the tested flash memory in program counter units
        intelHex (IntelHex16bit): Extracted data from hex by the IntelHex module.
    Returns:
        bytearray: extraced byte array data, prepared for CRC calculation
    """    
    assert (baseAddr % 2) == 0,"Base address must be even. See dsPIC architecture program counter. See dsPIC architecture and classB lib CRC documentation."
    assert (length % 2) == 0,"Length must be specified as program counter's length. So must be multiple of 2. See dsPIC architecture and classB lib CRC documentation."

    byteLength = length + length//2 #Convert dsPIC program counts to byte count (24bit)
    print("ByteLength:", byteLength)
    crcInput = bytearray(byteLength) 
    byteAddr = 0;

    for addr in range(length): # Convert data according to the ClassB CRC implementation

        #check upper or higher,lower address [Upper,Higher,Lower => program word of dsPIC]
        if (addr % 2) == 0: #even means Higher and lower
            crcInput[byteAddr+1] = (intelHex[baseAddr + addr] >> 8 )& 0xFF;    #Get Higher
            crcInput[byteAddr+2] = intelHex[baseAddr + addr] & 0xFF; #Get Lower

            #print("Addr:", hex(baseAddr+addr), "Higher:", crcInput[byteAddr+1], "Lower:", crcInput[byteAddr+2])

        else: # odd addres mean Upper byte
            crcInput[byteAddr] = intelHex[baseAddr + addr]; 
            #print("Addr:", hex(baseAddr+addr), "Upper:", crcInput[byteAddr])
            byteAddr = byteAddr+3 #increase byteAddress as this was the 
        
    
    return crcInput

Finally calculate the CRC for the required range

In [9]:
crcInput = ExtractHexData(0x250,0x6,ih)
crcResult = crc16_func(crcInput)
print("CRC Result:", hex(crcResult))

ByteLength: 9
CRC Result: 0xe8b0
